# Importing libraries

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Data Cleaning and Preprocessing

In [ ]:
# Connect to SQLite database
conn = sqlite3.connect('films.db')
# Querying the database

#Here we convert from sql tables to pandas tables
query = "SELECT * FROM title_ratings WHERE tconst IN (SELECT tb.tconst FROM title_basics AS tb WHERE tb.titleType = 'movie')"
df_title_ratings = pd.read_sql_query(query, conn)
df_title_ratings.replace("\\N", np.nan, inplace=True)

query = "SELECT nconst, primaryName FROM name_basics"
df_name_basics = pd.read_sql_query(query, conn)
df_name_basics.replace("\\N", np.nan, inplace=True)

query = "SELECT titleid, COUNT(CASE WHEN isOriginalTitle = 0 THEN 1 END) AS pocet_prekladov FROM title_akas GROUP BY titleid"
df_title_akas = pd.read_sql_query(query, conn)
df_title_akas.replace("\\N", np.nan, inplace=True)

query = "SELECT tconst, primaryTitle, isAdult, startYear, runtimeMinutes, genres  FROM title_basics WHERE titleType = 'movie'"
df_title_basics = pd.read_sql_query(query, conn)
df_title_basics.replace("\\N", np.nan, inplace=True)


query = """SELECT tconst, nconst, category 
            FROM title_principals 
            WHERE (category = 'actor' OR category = 'director') 
            AND tconst IN (SELECT tb.tconst FROM title_basics AS tb WHERE tb.titleType = 'movie')"""
df_title_principals = pd.read_sql_query(query, conn)
# Close the connection
conn.close()

#additional datasources 
df_top_actors = pd.read_csv('top20actors.csv')
df_best_directors = pd.read_csv('best_directors.csv')
df_worst_directors = pd.read_csv('worst_directors.csv')
df_disability = pd.read_csv('movies_disability.csv')

In [ ]:
df_title_principals.head()

In [ ]:
df_title_ratings.head()

In [ ]:
df_title_akas.head()

In [ ]:
#Here we create a year_diff
df_title_basics['year_diff'] = df_title_basics['startYear']
df_title_basics['year_diff'] = df_title_basics['year_diff'].apply(lambda x: 2024 - int(x) if pd.notna(x) else np.nan)
df_title_basics.head()

### Defining "oblubenost" (score)

In [ ]:
#log_score = avgRating*log(numvotes)
#norm_score = (avgRating/max(avgRating))*(numVotes/max(numVotes))
#prag_score = (avgRating*numVotes)/(avg(avgRating)*(avg(numVotes)))
df_title_ratings['log_score'] = df_title_ratings['averageRating'] * np.log(df_title_ratings['numVotes'])
df_title_ratings['norm_score'] = (df_title_ratings['averageRating'] / max(df_title_ratings['averageRating'])) * (df_title_ratings['numVotes'] / max(df_title_ratings['numVotes']))
df_title_ratings['prag_score'] = (df_title_ratings['averageRating'] * df_title_ratings['numVotes']) / (np.mean(df_title_ratings['averageRating']) * np.mean(df_title_ratings['numVotes'])) 

In [ ]:
df_title_ratings.describe()

In [ ]:
df = df_title_ratings[df_title_ratings['log_score'] > 130]
df.sort_values(by=['log_score'])

In [ ]:
df_title_ratings = df_title_ratings.drop('norm_score', axis=1)
df_title_ratings = df_title_ratings.drop('prag_score', axis=1)

In [ ]:
df_title_basics.head()

### Creating a bigger unified table "df_films"

In [ ]:
#here we join title_rating with title_akas on tconst
df_films = df_title_ratings.set_index('tconst').join(df_title_akas.set_index('titleId'), how='inner')
df_films = df_films.join(df_title_basics.set_index('tconst'), how='inner')

df_films['startYear'] = df_films['startYear'].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df_films['runtimeMinutes'] = df_films['runtimeMinutes'].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df_films['isAdult'] = df_films['isAdult'].apply(lambda x: int(x) if pd.notna(x) else np.nan)

df_films.head()

In [ ]:
df_films.columns

In [ ]:
df_films.describe()

### Creating a table for genre combinations

In [ ]:
df_genre_log = df_films.loc[:, ['primaryTitle', 'averageRating', 'numVotes', 'log_score', 'genres']]
df_genre_log['genres'] = df_genre_log['genres'].str.split(',')
df_genre_log['genres'] = df_genre_log['genres'].apply(lambda x: ",".join(sorted(x)) if isinstance(x, list) else np.nan)
df_genre_log.head()

In [ ]:
df_genre_log['genres'].value_counts()

In [ ]:
# Filter rows where 'genres' contains a comma
df_genre_combinations = df_genre_log[df_genre_log['genres'].str.contains(",", na=False)]
df_genre_count = df_genre_combinations['genres'].value_counts()
df_genre_combinations = list(df_genre_count[df_genre_count >= 1000].index)

In [ ]:
df_filtered_genres = df_genre_log[df_genre_log['genres'].isin(df_genre_combinations)]
df_filtered_genres

### One-hot encoding

In [ ]:
df_one_hot = pd.get_dummies(df_filtered_genres, columns=['genres'], prefix='', prefix_sep='')
df_first_part = df_one_hot.iloc[:, :2].reset_index() 
df_second_part = df_one_hot.iloc[:, 2:].reset_index()   

df_first_grouped = df_first_part.groupby('index', as_index=False).first()

df_second_grouped = df_second_part.groupby('index', as_index=False).sum()

df_filtered_genres = pd.concat([df_first_grouped, df_second_grouped], axis=1)

df_filtered_genres = df_filtered_genres.loc[:,~df_filtered_genres.columns.duplicated()]
df_filtered_genres = df_filtered_genres.set_index('index')

df_filtered_genres.head()

### Making columns from genres in table "db_films"

In [ ]:
# Split and explode the 'genres' column
df_films['genres'] = df_films['genres'].str.split(',')
df_exploded = df_films.explode('genres')

df_one_hot = pd.get_dummies(df_exploded, columns=['genres'], prefix='', prefix_sep='')
# Split the DataFrame into two parts
df_first_part = df_one_hot.iloc[:, :9].reset_index()  
df_second_part = df_one_hot.iloc[:, 9:].reset_index() 

# Apply groupby and first on the first part
df_first_grouped = df_first_part.groupby('index', as_index=False).first()

# Apply groupby and sum on the second part
df_second_grouped = df_second_part.groupby('index', as_index=False).sum()

# Concatenate the two DataFrames along the columns axis
df_films = pd.concat([df_first_grouped, df_second_grouped], axis=1)

# Dropping duplicate columns (as 'index' will be duplicated)
df_films = df_films.loc[:,~df_films.columns.duplicated()]
df_films.rename(columns={'index': 'tconst'}, inplace=True)
#df_films = df_films.set_index('index')

df_films.head()

# Preparing data with actors

In [ ]:
#here we separate best actors from all actors
df_top_actors['nconst']
df_films_with_top_actors = df_title_principals[df_title_principals['nconst'].isin(df_top_actors['nconst'])]
df_films_with_top_actors = df_films_with_top_actors[df_films_with_top_actors['category'] != 'director']
df_films_with_top_actors['category'].value_counts()


In [ ]:
#here we separate necessary values
merged_df = pd.merge(df_films_with_top_actors, df_top_actors, on='nconst', how='left')

result_df = merged_df.drop('nconst', axis=1)
result_df = result_df.drop('category', axis=1)

result_df = pd.merge(result_df, df_films.loc[:, ['tconst', 'averageRating','numVotes', 'log_score']], on='tconst', how='left')

result_df.head()

In [ ]:
#One-hot encoding
df_actors = pd.get_dummies(result_df, columns=['name'], prefix='', prefix_sep='')
df_actors.head()

### Preparing data for directors (best and worst)

In [ ]:
#best directors
df_best_directors = df_best_directors.iloc[:17, :]
df_merges = df_title_principals[df_title_principals['nconst'].isin(df_best_directors['nconst'])]
print(df_merges['category'].value_counts())
df_merges = df_merges[df_merges['category'] != 'actor']
df_merges.head()

merged_best_directors = pd.merge(df_merges, df_films.loc[:, ['tconst', 'log_score']], on='tconst', how='inner')
#worst directors
df_merges = df_title_principals[df_title_principals['nconst'].isin(df_worst_directors['nconst'])]
print(df_merges['category'].value_counts())
print(df_merges[df_merges['nconst'].isin(['nm0000219', 'nm1382072', 'nm3718935'])])
df_merges = df_merges[df_merges['category'] != 'actor']
df_merges.head()

merged_worst_directors = pd.merge(df_merges, df_films.loc[:, ['tconst', 'log_score']], on='tconst', how='inner')

#df_films = df_films.set_index('tconst')

In [ ]:
merged_best_directors.head()

In [ ]:
merged_worst_directors.head()

In [ ]:
#Getting avg_log_score from each of best and worst directors
avg_ls_best = merged_best_directors.groupby('nconst')['log_score'].mean().reset_index()
avg_ls_best.rename(columns={'log_score': 'avg_log_score'}, inplace=True)
print(avg_ls_best)

avg_ls_worst = merged_worst_directors.groupby('nconst')['log_score'].mean().reset_index()
avg_ls_worst.rename(columns={'log_score': 'avg_log_score'}, inplace=True)

avg_ls_worst

In [ ]:
print(df_worst_directors[~df_worst_directors['nconst'].isin(avg_ls_worst['nconst'])])

# Exploratory Data Analysis (EDA)

In [ ]:
df_films = df_films.set_index('tconst')
df_films.describe()

In [ ]:
print(df_films.iloc[:, 9:].columns.tolist())

In [ ]:
# Basic data exploration
print(df_films.shape)
print(df_films.dtypes)

In [ ]:
df_films.isna().sum().sort_values(ascending=False)

In [ ]:
# Univariate Analysis
fig, ax = plt.subplots()
df_films['log_score'].hist(bins = 50)
ax.set(xlabel="Log skóre", ylabel="Počet filmov", title="Rozdelenie log skóre")
plt.show()

fig, ax = plt.subplots()
sns.boxplot(y='log_score', data=df_films)
ax.set(ylabel="Log skóre", title="Rozdelenie log skóre")
plt.show()

In [ ]:
categorical_columns = ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 
                       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 
                       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 
                       'Sci-Fi', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']

category_counts = df_films[categorical_columns].sum()

category_counts_sorted = category_counts.sort_values(ascending=False)

# Ploting the counts in a single bar plot
category_counts_sorted.plot(kind='bar', figsize=(15, 7))
plt.title('Počet filmov v jednotlivých kategóriách')
# plt.xlabel('Category')
# plt.ylabel('Count')
plt.show()

In [ ]:
import plotly.express as px

genre_counts = df_exploded.groupby(['startYear', 'genres']).size().reset_index(name='count')


fig = px.line(genre_counts, x='startYear', y='count', color='genres', title='Number of Films per Genre Over Years')
fig.show()


In [ ]:
# Bivariate Analysis
sns.scatterplot(x='startYear', y='log_score', data=df_films)
plt.show()

In [ ]:
columns = list(categorical_columns)
columns = ['averageRating', 'numVotes', 'log_score'] + columns
df_film_genres = df_films.loc[:, columns]
df_film_genres.head()

In [ ]:
corr_matrix = df_film_genres.corr(numeric_only = True)
corr_matrix

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
sns.heatmap(corr_matrix.iloc[3:, :3], vmin=-1, vmax=1)
ax.set_title("Korelácia žánru so skóre")
ax.set_xticklabels(["Priemerné skóre", "Počet hlasov", "Log skóre"])
plt.show()

In [ ]:
print(corr_matrix["log_score"].iloc[3:].sort_values(ascending=False).describe().to_markdown()) #.describe().to_markdown()

In [ ]:
corr_matrix = df_filtered_genres.corr(numeric_only = True)
corr_matrix

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
sns.heatmap(corr_matrix.iloc[3:, :3], vmin=-1, vmax=1)
ax.set_title("Korelácia skupiny žánrov so skóre")
ax.set_xticklabels(["Priemerné skóre", "Počet hlasov", "Log skóre"])
plt.show()


In [ ]:
print(corr_matrix["log_score"].iloc[3:].sort_values(ascending=False).head(10).to_markdown()) #.describe().to_markdown()

In [ ]:
df_film_stats = df_films.iloc[:, :9]
df_film_stats.head()

In [ ]:
# Correlation Matrix
corr_matrix = df_film_stats.corr(numeric_only = True)
corr_matrix

In [ ]:
sns.heatmap(corr_matrix[["log_score"]], vmin=-1, vmax=1 )
plt.show()

In [ ]:
sns.scatterplot(x='runtimeMinutes', y='log_score', 
                data=df_film_stats[(df_film_stats['runtimeMinutes'] < 480)])
plt.show()

In [ ]:
# Pair Plot
sns.pairplot(df_film_stats[(df_film_stats['runtimeMinutes'] < 480)], diag_kind='kde')
plt.show()

# Statistical Analysis

In [ ]:
from scipy import stats

### Functions

In [ ]:
def test_normality(data):
    if len(data) <= 5000:
        stat, p_value = stats.shapiro(data)
        print(f'Shapiro-Wilk Test: Statistics={stat}, p-value={p_value}')
    else:
        stat, p_value = stats.kstest(data, 'norm', args=(np.mean(data), np.std(data)))
        print(f'Kolmogorov-Smirnov Test: Statistics={stat}, p-value={p_value}')
    if p_value > 0.05:
        print("Our data is most likely from Normal distribution.")
    else:
        print("Our data is not from Normal distribution.")

def qq_plot(data, title):
    stats.probplot(data, dist="norm", plot=plt)
    plt.title(title)
    plt.show()
    
def histogram(data):
    fig, ax1 = plt.subplots()

    sns.histplot(data, ax=ax1, color='blue', alpha=0.5)
    ax1.set_xlabel("Score")
    ax1.set_ylabel("Frequency", color='blue')

    ax2 = ax1.twinx()
    sns.kdeplot(data, ax=ax2, color='red', linewidth=2)
    ax2.set_ylabel("KDE", color='red')

    ax1.grid(axis='y', linestyle='--', alpha=0.6)
    ax2.grid(False)

    plt.title("Score Distribution with KDE")
    plt.show()
       
def hypothesis_test(x, y, alternative):
    if stats.shapiro(x)[1] > 0.05 and stats.shapiro(y)[1] > 0.05:
        # IF both groups are normally distributed then we use student's t-test
        stat, p = stats.ttest_ind(x, y, alternative=alternative)
        print(f'T-test: Statistics={stat}, p-value={p}')
    else:
        # IF at least one group is not normally distributed we use Mann-Whitney U test
        stat, p = stats.mannwhitneyu(x, y, alternative=alternative)
        print(f'Mann-Whitney U Test: Statistics={stat}, p-value={p}')

    if p > 0.05:
        print("Fail to reject the null hypothesis (H0)")
    else:
        print("Reject the null hypothesis (H0), accept the alternative hypothesis (H1)")

## Directors testing

### Normality testing 

In [ ]:
x = np.array(avg_ls_best.loc[:, 'avg_log_score'])  # Best directors avg_log_score
y = np.array(avg_ls_worst.loc[:, 'avg_log_score'])  # Worst directors avg_log_score
print(x.shape, y.shape)

In [ ]:
histogram(x)
histogram(y)

In [ ]:
print("Testing normality for x:")
test_normality(x)
print("\nTesting normality for y:")
test_normality(y)

qq_plot(x, "QQ Plot for x")
qq_plot(y, "QQ Plot for y")

### Hypothesis testing (H0: μx <= μy vs. H1: μx > μy)

In [ ]:
hypothesis_test(x, y, 'greater')

## Translation testing

### Normality testing

In [ ]:
preklad_median = np.median(df_film_stats['pocet_prekladov'])
print(preklad_median)

In [ ]:
#median prekladov = 3
malo_prekladov = df_film_stats[df_film_stats['pocet_prekladov'] <= 3]
vela_prekladov = df_film_stats[df_film_stats['pocet_prekladov'] > 3]
print(malo_prekladov.shape, vela_prekladov.shape)

In [ ]:
histogram(malo_prekladov['log_score'])
histogram(vela_prekladov['log_score'])

In [ ]:
print("Testing normality for x:")
test_normality(malo_prekladov['log_score'])
print("\nTesting normality for y:")
test_normality(vela_prekladov['log_score'])

qq_plot(malo_prekladov['log_score'], "QQ Plot for x")
qq_plot(vela_prekladov['log_score'], "QQ Plot for y")

### Hypothesis testing (H0: μx >= μy vs. H1: μx < μy)

In [ ]:
hypothesis_test(malo_prekladov['log_score'], vela_prekladov['log_score'], 'less')

## Visualisation and analysis

### Functions

In [ ]:
def diff_of_avg_log(data, analyzed_data, title):
    
    data_mean = np.mean(data['log_score'])
    data = data.reset_index()
    data.rename(columns={'index': 'tconst'}, inplace=True)
    excluded_data = data[~data['tconst'].isin(analyzed_data['tconst'])]
    excluded_data_mean = np.mean(excluded_data['log_score'])
    
    diff = excluded_data_mean - data_mean
    result = - round(diff / data_mean * 100, 2)
    
    plt.figure(figsize=(10, 6))

    # Plotting the first dataset
    sns.scatterplot(x='startYear', y='log_score', data=data, hue=0.5, label='All data')

    # Plotting the second dataset
    sns.scatterplot(x='startYear', y='log_score', data=data[data['tconst'].isin(analyzed_data['tconst'])], 
                    color='red', label='Excluded data')

    plt.axhline(y=data_mean, color='blue', linestyle='--', label='Threshold')

    # Adding labels and title
    plt.xlabel('Start Year')
    plt.ylabel('Log Score')
    plt.title(title)

    # Display the plot
    plt.legend()
    plt.show()
    
    if diff >= 0:
        return print(f"Analyzed movies had negative or no impact on the mean log_score of the whole data.\nThe significance of the impact is {result}%.")
    else:
        return print(f"Analyzed movies had a positive impact on the mean log_score of the whole data.\nThe significance of the impact is {result}%.")


### Greatest Actors

In [ ]:
diff_of_avg_log(df_film_stats, df_films_with_top_actors, 'Movies with the greatest Actors')

### Disabilities

In [ ]:
diff_of_avg_log(df_film_stats, df_disability, 'Movies in which the main character has a disability')

### Adults

In [ ]:
adult_data = df_film_stats[df_film_stats['isAdult'] == 1].reset_index()
adult_data.rename(columns={'index': 'tconst'}, inplace=True)
diff_of_avg_log(df_film_stats, adult_data, 'Movies which are for adults')